# Table question answering

In [13]:
from transformers import pipeline, TapasTokenizer, TapasForQuestionAnswering

Now, to extract the answer from a table, we need table question answering \
A model that seems recommended in this area is the google model TAPAS \
Due to time concerns, I am using one that is pretrained on a Wikipedia dataset \
Ideally in the future, I learn how to train this one once I have the database structure perfectly defined

In [14]:
tokenizer = TapasTokenizer.from_pretrained("google/tapas-base-finetuned-wtq")
model = TapasForQuestionAnswering.from_pretrained("google/tapas-base-finetuned-wtq")


tokenizer_config.json: 100%|██████████| 490/490 [00:00<00:00, 763kB/s]
vocab.txt: 100%|██████████| 262k/262k [00:00<00:00, 1.65MB/s]
special_tokens_map.json: 100%|██████████| 154/154 [00:00<00:00, 364kB/s]
config.json: 100%|██████████| 1.66k/1.66k [00:00<00:00, 3.24MB/s]
pytorch_model.bin: 100%|██████████| 443M/443M [00:51<00:00, 8.66MB/s] 


Creating pipeline for our model

In [15]:
pipe = pipeline("table-question-answering", model = model, tokenizer = tokenizer)

In [16]:
import pandas as pd

Let's bring back that table from the psql notebook

In [17]:
table = pd.read_csv("psqltable.csv", index_col=0)
table

,career,coordinator
0,Data Engineering,Pascual Icíar
1,Robotics,Jerónimo Micaela
2,Embedded Systems,Alma Irma
3,Cybersecurity,José Ángel Anastacia


In [18]:
query = "Who is the coordinator of the Data Engineering career?"

We need both a prompt and a table for the model to give us a result

In [19]:
pipe(table = table, query = query)

/home/gabriel/Documentos/finalproject/nlpupy_env/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2762: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/home/gabriel/Documentos/finalproject/nlpupy_env/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1561: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


{'answer': 'Pascual Icíar',
 'coordinates': [(0, 1)],
 'cells': ['Pascual Icíar'],
 'aggregator': 'NONE'}

In [30]:
query1 = "Trying to find the coordinator of cybersecurity"

In [31]:
pipe(table = table, query = query1)

{'answer': 'José Ángel Anastacia',
 'coordinates': [(3, 1)],
 'cells': ['José Ángel Anastacia'],
 'aggregator': 'NONE'}

On further testing, it works decently well surprisingly, but loses a bit more nuance of the human language \
It is very specific with the column names sometimes, and cannot detect synonyms or their verb forms sometimes \
In the future, I would like to train this with the defined database